# Breast cancer PSI-MS - Model update

Import packages

In [1]:
import os
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn import svm
from sklearn import model_selection
from sklearn.metrics import confusion_matrix, roc_curve
from sklearn import feature_selection
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import pickle

In [2]:
def performance(diagnosis,prediction):
    result=confusion_matrix(diagnosis,prediction)
    tn,fp,fn,tp=result.ravel()
    accuracy=(tp+tn)/np.sum(result)
    specificity=tn/sum(result[0,:])
    sensitivity=tp/sum(result[1,:])
    #print('Accuracy: %.3f Sensitivity: %.3f Specificity: %.3f'%(accuracy,sensitivity,specificity))
    return accuracy, specificity, sensitivity

def C_optimization(C_all, weight, x, y):
    result_c=[]
    for c in C_all:
        svc=svm.SVC(C=c, kernel='linear',class_weight=weight)
        cv_prediction=model_selection.cross_val_predict(svc,x,y,cv=5)
        accuracy, specificity, sensitivity=performance(y,cv_prediction)
        result_c.append([c, accuracy, specificity, sensitivity])

    df_results_c = pd.DataFrame(result_c, columns =['C','Accuracy', 'Specificity', 'Sensitivity'])
    df_results_c.sort_values(by='Accuracy',ascending=False, ignore_index=True,inplace=True)

    return df_results_c

def Feature_optimization(num_start, num_end, C, weight, x, y):
    result_feature=[]

    for i in range(num_start,num_end,10):
        svc=svm.SVC(C=C, kernel='linear',class_weight=weight)
        svc=feature_selection.RFE(svc,n_features_to_select=i)
        cv_prediction=model_selection.cross_val_predict(svc,x,y,cv=5)
        accuracy, specificity, sensitivity=performance(y,cv_prediction)
        result_feature.append([i, accuracy, specificity, sensitivity])
    df_results_feature = pd.DataFrame(result_feature, columns =['Features','Accuracy', 'Specificity', 'Sensitivity'])
    df_results_feature.sort_values(by=['Accuracy','Features'],ascending=[False,True], ignore_index=True, inplace=True)
    return df_results_feature

def Plot_C_optimization(df_results_c, c, filename):
    fig = go.Figure()

    # Plot trend
    fig.add_trace(go.Scatter(x=df_results_c['C'], y=df_results_c['Sensitivity']*100,name='Sensitivity',
                             line={'color':'#F4A016','width':5},marker={'size':8}))
    fig.add_trace(go.Scatter(x=df_results_c['C'], y=df_results_c['Specificity']*100,name='Specificity',
                             line={'color':'#157B3A','width':5},marker={'size':8}))
    fig.add_trace(go.Scatter(x=df_results_c['C'], y=df_results_c['Accuracy']*100,name='Accuracy',
                             line={'color':'#A63354','width':5},marker={'size':8}))
    fig.add_trace(go.Scatter(x=[c,c], y=[0,100], mode='lines',
                             line={'dash':'dash','color':'black','width':3},showlegend=False))

    # Some aesthetical adjustments to layout
    fig.update_xaxes(type="log")
    fig.update_layout(autosize=False,width=550,height=350,font=dict(size=14), template='simple_white',
                      legend=dict(yanchor="bottom", y=0.05,xanchor="right",x=0.6, bordercolor="Black",borderwidth=1))

    fig.write_image(os.path.join(Path,filename),scale=3)
    plt.close()

def Plot_Feature_optimization(df_results_feature, Feature, filename):
    fig = make_subplots(rows=2, cols=1, vertical_spacing = 0.15, row_heights=[3.5,1])
    cut_interval = [10, 70]

    # Plot trend
    fig.add_trace(go.Scatter(x=df_results_feature['Features'], y=df_results_feature['Sensitivity']*100,name='Sensitivity',
                        line={'color':'#F4A016','width':5},marker={'size':8}), row=1, col=1)
    fig.add_trace(go.Scatter(x=df_results_feature['Features'], y=df_results_feature['Specificity']*100,name='Specificity',
                        line={'color':'#157B3A','width':5},marker={'size':8}), row=1, col=1)
    fig.add_trace(go.Scatter(x=df_results_feature['Features'], y=df_results_feature['Accuracy']*100,name='Accuracy',
                        line={'color':'#A63354','width':5},marker={'size':8}), row=1, col=1)
    fig.add_trace(go.Scatter(x=[Feature,Feature], y=[0,100], mode='lines',
                         line={'dash':'dash','color':'black','width':3},showlegend=False), row=1, col=1)
    fig.add_trace(go.Scatter(x=[Feature,Feature], y=[0,100], mode='lines',
                         line={'dash':'dash','color':'black','width':3},showlegend=False), row=2, col=1)
    # Some aesthetical adjustments to layout
    fig.update_yaxes(range=[cut_interval[1], 100], row=1, col=1)
    fig.update_xaxes(range=[0,150],visible=False, row=1, col=1)
    fig.update_yaxes(range=[0, cut_interval[0]], row=2, col=1)
    fig.update_xaxes(range=[0,150],row=2, col=1)

    fig.update_layout(autosize=False,width=550,height=350,font=dict(size=14), template='simple_white',
                 legend=dict(yanchor="bottom", y=0.05,xanchor="right",x=1, bordercolor="Black",borderwidth=1))
    fig.write_image(os.path.join(Path,filename),scale=3)
    plt.close()

File directory / Filename

In [3]:
Path = r'D:\Breast Cancer PSIMS\Codes and figures'
Filename_train = 'PSIMS_Data_from_Lab.csv'
Filename_external = 'PSIMS_Data_OnSite_All.csv'

Preprocess data

In [4]:
# Read raw file for model construction
Raw_data = pd.read_csv(os.path.join(Path,Filename_train))
OnSite_Data = pd.read_csv(os.path.join(Path,Filename_external))

# Convert to malignant (1) and benign (0), ignoring subtyping
Raw_data['subtype'] = Raw_data['subtype'].apply(lambda x: np.clip(x,0,1))

# Random split into 2 to 8 (considering label distribution)
x_train_index,x_test_index,y_train,y_test=model_selection.train_test_split(
    Raw_data.index,Raw_data['subtype'],test_size=0.2,random_state=1,stratify=Raw_data['subtype'])

x_train = Raw_data.iloc[x_train_index,2:]
x_test = Raw_data.iloc[x_test_index,2:]

x_external = OnSite_Data.iloc[:,2:]
y_external = OnSite_Data.iloc[:,1]

Updating every 6 months

In [5]:
#C_all = [500,750,1000,1200,1500,2000,2500]
C_all = [1,5,10,25,125,250,750,1000,1500,2000,2500,5000,10000]

#Feature_num = [50,120]
Feature_num = [10,150]

In [6]:
Patient_info = pd.read_excel(os.path.join(Path,'Patient_Info_onsite.xlsx'),sheet_name="All")
Patient_info['Date'] = pd.to_datetime(Patient_info['Date'], format='%Y%m%d')
acc_cv, acc_ex, sen_cv, sen_ex, spe_cv, spe_ex, num_total = [], [0], [], [0] ,[] ,[0], []
Patient_num, Benign_num, Malignant_num = [0], [0], [0]

T = pd.date_range(start='2020-8-1', periods=5, freq='6M')
Weight = 'balanced'

for i in range(4):
    if i == 0:
        svc=svm.SVC(C=1500, kernel='linear',class_weight={0:0.3,1:0.7})
        svc=feature_selection.RFE(svc,n_features_to_select=60)
        cv_prediction=model_selection.cross_val_predict(svc,x_train,y_train,cv=5)
        print('5-fold cross validation results:')
        acc, spe, sen=performance(y_train,cv_prediction)
        print('Accuracy: %.3f Sensitivity: %.3f Specificity: %.3f'%(acc,sen,spe))
        
        acc_cv.append(acc)
        sen_cv.append(sen)
        spe_cv.append(spe)
        
        svc=svc.fit(x_train,y_train)
        num_total.append(len(y_train))
        
        mask = (Patient_info['Date'] > T[i]) & (Patient_info['Date'] <= T[i+1])
        mask_pos = (Patient_info['Type'] == 1)
        mask_neg = (Patient_info['Type'] == 0)
        Patient_num.append(sum(mask))
        Benign_num.append(sum(mask & mask_neg))
        Malignant_num.append(sum(mask & mask_pos))
        
        validation_prediction=svc.predict(x_external.loc[mask,:])
        print('Validation set results:')
        acc, spe, sen=performance(y_external.loc[mask],validation_prediction)
        print('Accuracy: %.3f Sensitivity: %.3f Specificity: %.3f'%(acc,sen,spe))
        
        acc_ex.append(acc)
        sen_ex.append(sen)
        spe_ex.append(spe)
        
        New_X = x_train.append(x_external.loc[mask,:])
        New_Y = y_train.append(y_external.loc[mask])
        print('-'*50)
        
    else:
        mask = (Patient_info['Date'] > T[i]) & (Patient_info['Date'] <= T[i+1])
        mask_pos = (Patient_info['Type'] == 1)
        mask_neg = (Patient_info['Type'] == 0)
        Patient_num.append(sum(mask))
        Benign_num.append(sum(mask & mask_neg))
        Malignant_num.append(sum(mask & mask_pos))
        
        # Optimize model
        df_results_c = C_optimization(C_all, Weight, New_X, New_Y)
        c = df_results_c.loc[0,'C']
        print('Optimized C: '+str(c))
        df_results_c.sort_values(by='C',ascending=True, inplace=True)
        Plot_C_optimization(df_results_c, c, 'C optimization_'+str(i)+'.png')
        
        df_results_feature = Feature_optimization(Feature_num[0], Feature_num[1], c, Weight, New_X, New_Y)
        Feature = df_results_feature.loc[0,'Features']
        print('Optimized number of features: '+str(Feature))
        df_results_feature.sort_values(by='Features',ascending=True, inplace=True)
        Plot_Feature_optimization(df_results_feature, Feature, 'N feature optimization_'+str(i)+'.png')
        
        # 5-fold cross validation
        svc=svm.SVC(C=c, kernel='linear',class_weight=Weight)
        svc=feature_selection.RFE(svc,n_features_to_select=Feature)
        cv_prediction=model_selection.cross_val_predict(svc,New_X,New_Y,cv=5)
        print('5-fold cross validation results:')
        acc, spe, sen=performance(New_Y,cv_prediction)
        print('Accuracy: %.3f Sensitivity: %.3f Specificity: %.3f'%(acc,sen,spe))
        acc_cv.append(acc)
        sen_cv.append(sen)
        spe_cv.append(spe)
        
        svc=svc.fit(New_X,New_Y)
        num_total.append(len(New_Y))
        
        mask2 = (Patient_info['Date'] > T[i]) & (Patient_info['Date'] <= T[i+1])
        validation_prediction=svc.predict(x_external.loc[mask,:])
        print('Validation set results:')
        acc, spe, sen=performance(y_external.loc[mask],validation_prediction)
        print('Accuracy: %.3f Sensitivity: %.3f Specificity: %.3f'%(acc,sen,spe))
        acc_ex.append(acc)
        sen_ex.append(sen)
        spe_ex.append(spe)
        
        if i==3:
            target_id = mask
            target_ans = y_external.loc[mask]
            target_pred = validation_prediction
        
        New_X = New_X.append(x_external.loc[mask,:])
        New_Y = New_Y.append(y_external.loc[mask])
        print('-'*50)

print('Final model')
New_X = x_train.append(x_external)
New_Y = y_train.append(y_external)

# Optimize model
df_results_c = C_optimization(C_all, Weight, New_X, New_Y)
c = df_results_c.loc[0,'C']
print('Optimized C: '+str(c))
df_results_c.sort_values(by='C',ascending=True, inplace=True)
Plot_C_optimization(df_results_c, c, 'C optimization_final.png')

df_results_feature = Feature_optimization(Feature_num[0], Feature_num[1], c, Weight, New_X, New_Y)
Feature = df_results_feature.loc[0,'Features']
print('Optimized number of features: '+str(Feature))
df_results_feature.sort_values(by='Features',ascending=True, inplace=True)
Plot_Feature_optimization(df_results_feature, Feature, 'N feature optimization_final.png')

# 5-fold cross validation
svc=svm.SVC(C=c, kernel='linear',class_weight=Weight)
svc=feature_selection.RFE(svc,n_features_to_select=Feature)
cv_prediction=model_selection.cross_val_predict(svc,New_X,New_Y,cv=5)
print('5-fold cross validation results:')
acc, spe, sen=performance(New_Y,cv_prediction)
print('Accuracy: %.3f Sensitivity: %.3f Specificity: %.3f'%(acc,sen,spe))
acc_cv.append(acc)
sen_cv.append(sen)
spe_cv.append(spe)
num_total.append(len(New_Y))
print('Finished')

5-fold cross validation results:
Accuracy: 0.875 Sensitivity: 0.805 Specificity: 0.903
Validation set results:
Accuracy: 0.870 Sensitivity: 0.750 Specificity: 0.924
--------------------------------------------------
Optimized C: 1500
Optimized number of features: 70
5-fold cross validation results:
Accuracy: 0.900 Sensitivity: 0.805 Specificity: 0.940
Validation set results:
Accuracy: 0.843 Sensitivity: 0.871 Specificity: 0.833
--------------------------------------------------
Optimized C: 5000
Optimized number of features: 60
5-fold cross validation results:
Accuracy: 0.896 Sensitivity: 0.806 Specificity: 0.932
Validation set results:
Accuracy: 0.881 Sensitivity: 0.883 Specificity: 0.880
--------------------------------------------------
Optimized C: 750
Optimized number of features: 60
5-fold cross validation results:
Accuracy: 0.875 Sensitivity: 0.815 Specificity: 0.902
Validation set results:
Accuracy: 0.840 Sensitivity: 0.778 Specificity: 0.875
-----------------------------------

Plot results

In [7]:
T = pd.date_range(start='2020-8-1', periods=5, freq='6M')
df_Time_Track = pd.DataFrame({'Date':T, 'Accumulated Number':num_total,'Patient number':Patient_num,
                              'Benign number':Benign_num,'Malignant number':Malignant_num,
                              'Accuracy_CV':acc_cv, 'Sensitivity_CV':sen_cv, 'Specificity_CV':spe_cv,
                             'Accuracy_ex':acc_ex, 'Sensitivity_ex':sen_ex, 'Specificity_ex':spe_ex})

In [9]:
fig = go.Figure()

# Plot cross validation
fig.add_trace(go.Scatter(x=df_Time_Track['Date']+pd.DateOffset(days=10), y=df_Time_Track['Accuracy_CV']*100,name='Cross valition',
                            mode="lines+markers",line={'color':'#A63354','width':5},marker={'size':10}))
# Plot external validation
fig.add_trace(go.Scatter(x=df_Time_Track.loc[1:4,'Date'], y=df_Time_Track.loc[1:4,'Accuracy_ex']*100,name='External validation',
                        marker={'size':8}, line={'color':'#A63354','width':3,'dash': 'dashdot'}))
fig.update_layout(autosize=False,width=900,height=350,font=dict(size=24),
                  legend=dict(yanchor="bottom", y=0,xanchor="right",x=1),template='simple_white')
fig.update_yaxes(range=[50, 100])
fig.show()
fig.write_image(os.path.join(Path,'Model update_Accuracy.png'),scale=3)
plt.close()

In [10]:
fig = go.Figure()

# Plot cross validation
fig.add_trace(go.Scatter(x=df_Time_Track['Date']+pd.DateOffset(days=10), y=df_Time_Track['Specificity_CV']*100,name='Cross valition',
                            mode="lines+markers",line={'color':'#157B3A','width':5},marker={'size':10}))
# Plot external validation
fig.add_trace(go.Scatter(x=df_Time_Track.loc[1:4,'Date'], y=df_Time_Track.loc[1:4,'Specificity_ex']*100,name='External validation',
                        marker={'size':8}, line={'color':'#157B3A','width':3,'dash': 'dashdot'}))
fig.update_layout(autosize=False,width=900,height=350,font=dict(size=24),
                  legend=dict(yanchor="bottom", y=0,xanchor="right",x=1),template='simple_white')
fig.update_yaxes(range=[50, 100])
fig.show()
fig.write_image(os.path.join(Path,'Model update_Specificity.png'),scale=3)
plt.close()

In [12]:
fig = go.Figure()

# Plot cross validation
fig.add_trace(go.Scatter(x=df_Time_Track['Date']+pd.DateOffset(days=10), y=df_Time_Track['Sensitivity_CV']*100,name='Cross valition',
                            mode="lines+markers",line={'color':'#F4A016','width':5},marker={'size':10}))
# Plot external validation
fig.add_trace(go.Scatter(x=df_Time_Track.loc[1:4,'Date'], y=df_Time_Track.loc[1:4,'Sensitivity_ex']*100,name='External validation',
                        marker={'size':8}, line={'color':'#F4A016','width':3,'dash': 'dashdot'}))
fig.update_layout(autosize=False,width=900,height=350,font=dict(size=24),
                  legend=dict(yanchor="bottom", y=0,xanchor="right",x=1),template='simple_white')
fig.update_yaxes(range=[45, 95])
fig.show()
fig.write_image(os.path.join(Path,'Model update_Sensitivity.png'),scale=3)
plt.close()

In [15]:
fig = go.Figure()

# Plot trend
fig.add_trace(go.Scatter(x=df_Time_Track['Date']+pd.DateOffset(days=10), y=df_Time_Track['Accumulated Number'],name='Number of accumulated samples',
                         mode="lines+markers+text",line={'color':'black','width':5},marker={'size':10},
                         text=df_Time_Track['Accumulated Number'],textposition="top center"))

# Plot population
fig.add_trace(go.Bar(x=df_Time_Track.loc[1:4,'Date'], y=df_Time_Track.loc[1:4,'Benign number'],name='Number of new benign samples',marker_color='#157B3A',opacity=0.8))
fig.add_trace(go.Bar(x=df_Time_Track.loc[1:4,'Date'], y=df_Time_Track.loc[1:4,'Malignant number'],name='Number of new malignant samples',
                     text=df_Time_Track.loc[1:4,'Patient number'],textposition='outside',marker_color='#F4A016',opacity=0.8))
fig.update_layout(barmode='stack')

fig.update_yaxes(range=[0,900])

fig.update_layout(autosize=False,width=1200,height=500,font=dict(size=24), template='simple_white',
                 legend=dict(orientation='v',yanchor="top", y=1,xanchor="left",x=0,font = dict(size = 24)))

fig.show()
fig.write_image(os.path.join(Path,'Model update_Sample.png'),scale=3)
plt.close()

In [16]:
df_Time_Track

,Date,Accumulated Number,Patient number,Benign number,Malignant number,Accuracy_CV,Sensitivity_CV,Specificity_CV,Accuracy_ex,Sensitivity_ex,Specificity_ex
0,2020-08-31,144,0,0,0,0.875000,0.804878,0.902913,0.000000,0.000000,0.000000
1,2021-02-28,259,115,79,36,0.891892,0.805195,0.928571,0.869565,0.750000,0.924051
2,2021-08-31,374,115,84,31,0.893048,0.805556,0.928571,0.843478,0.870968,0.833333
3,2022-02-28,534,160,100,60,0.874532,0.815476,0.901639,0.875000,0.883333,0.870000
4,2022-08-31,684,150,96,54,0.877193,0.774775,0.926407,0.840000,0.777778,0.875000
